In [21]:
import pandas as pd
import pic

import dask.dataframe as ddf

import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)-8s [%(lineno)d] %(message)s')

ModuleNotFoundError: No module named 'dask'

In [17]:
def read_mazsola():
    with open('/mnt/permanent/Language/Hungarian/Dic/sass15-535k-igei-szerkezet/mazsola_adatbazis.txt') as infile:
        for i, line in enumerate(infile):
            if i == 100000:
                break
            if not i % 1000000:
                logging.info('{:.0%}'.format(i/27970403))
            line = line.strip()
            if not line:
                continue
            sep = '@@'
            yield(dict(tuple(token.split(sep, maxsplit=1)) 
                       for token in filter(lambda token: '@@' in token, line.split())))

In [ ]:

data = ddf.read_csv('test.csv')

This data object hasn't actually done anything at this point; it just contains a "recipe" of sorts to read the dataframe from disk in manageable chunks. If you want to materialize the data, you can call compute():

df = data.compute().reset_index(drop=True)

At this point, you have a standard pandas dataframe (we call reset_index because by default each partition is independently indexed). The result is equivalent to what you get by calling pd.read_csv directly:

df.equals(pd.read_csv('test.csv'))
# True

The benefit of dask is you can add instructions to this "recipe" for constructing your dataframe; for example, you could make each partition of the data sparse as follows:

data = data.map_partitions(lambda part: part.to_sparse(fill_value=0))

At this point, calling compute() will construct a sparse array:

df = data.compute().reset_index(drop=True)
type(df)

In [18]:
mazsola = pd.DataFrame.from_records(read_mazsola())

INFO     [7] 0%


In [20]:
mazsola.columns

Index(['', 'ABL', 'ACC', 'ADE', 'ALL', 'CAU', 'DAT', 'DEL', 'ELA', 'ESS',
       'FAC', 'FOR', 'ILL', 'INE', 'INS', 'KÖZÖTT', 'Közel', 'Mellett', 'NOM',
       'Pe3', 'SUB', 'SUP', 'TEM', 'TER', 'alatt', 'alá', 'alól', 'belül',
       'belől', 'dacára', 'együtt', 'ellen', 'ellenére', 'elé', 'elébe',
       'elől', 'előtt', 'fele', 'felett', 'felé', 'felül', 'felől', 'fogva',
       'folytán', 'fölé', 'fölött', 'gyanánt', 'helyett', 'iránt', 'keresztül',
       'képest', 'kívül', 'köré', 'körül', 'körülött', 'következtében',
       'közben', 'közel', 'közelében', 'közt', 'közé', 'között', 'közül',
       'mellett', 'mellé', 'mellől', 'mentén', 'miatt', 'mögé', 'mögött',
       'múlva', 'nélkül', 'rajta', 'részére', 'révén', 'stem', 'szemben',
       'szerint', 'számára', 'túl', 'után', 'végett', 'Útján', 'által', 'óta',
       'útján'],
      dtype='object')